In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaPySparkIntegration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()


In [25]:
# Set Spark logging level to "ERROR" to reduce log output
spark.sparkContext.setLogLevel("ERROR")


In [26]:
topic_nm = "your_topic_name"
#topic_nm = "new_topic_name"

In [27]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_nm) \
    .load()




./kafka-console-producer.sh --broker-list localhost:9092 --topic my-topic --property "parse.key=true" -
-property "key.separator=:"

myKey1:Hello Kafka from PySpark
myKey2:Another message


In [28]:
from pyspark.sql.functions import col, from_json, schema_of_json


In [ ]:
# Define the schema for the JSON data
json_schema = "STRUCT<key: STRING, value: STRING>"

# Start a streaming query that writes to the console
query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
          .withColumn("parsed_value", from_json(col("value"), json_schema)) \
          .writeStream \
          .outputMode("append") \
          .format("console") \
          .start()

# Wait for the termination signal to keep the stream open
query.awaitTermination()


-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+------------+
| key|value|parsed_value|
+----+-----+------------+
|NULL|  b,a|{NULL, NULL}|
+----+-----+------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+------------+
| key|value|parsed_value|
+----+-----+------------+
|NULL|{a,b}|{NULL, NULL}|
+----+-----+------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----+------------+
| key|value|parsed_value|
+----+-----+------------+
|NULL| exit|{NULL, NULL}|
+----+-----+------------+

